# Thin-Film Heightfield Sandbox

> **Qualifier:** This notebook offers a qualitative toy model only. It helps visualize tendencies before pouring, but it is **not** a validated CFD or multiphysics simulation. Confirm everything on the bench.


## Model Overview

- `Φ_ac(x,y)` synthesizes a standing-wave potential as a sum of orthogonal sines.
- `Φ_mag(x,y)` places Gaussian magnets beneath the tray midline.
- The height field `h(x,y,t)` updates via explicit Euler with gentle smoothing to mimic surface tension.
- Drive weights `α(t)` and `β(t)` taper toward zero as the mix gels (slider controls the initial value).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

plt.rcParams["figure.figsize"] = (6, 5)

# Spatial grid (unit square proxy for 120 mm tray)
NX, NY = 80, 80
x = np.linspace(0.0, 1.0, NX)
y = np.linspace(0.0, 1.0, NY)
X, Y = np.meshgrid(x, y)


In [ ]:
def acoustic_potential(freq_x=2, freq_y=2, phase=0.0):
    """Standing wave proxy as orthogonal sines."""
    return np.sin(2 * np.pi * freq_x * X) * np.sin(2 * np.pi * freq_y * Y + phase)


def magnet_potential(magnet_centers=None, sigma=0.05):
    """Sum of Gaussian wells approximating magnet influence."""
    if magnet_centers is None:
        magnet_centers = [(0.3, 0.5), (0.7, 0.5)]
    phi = np.zeros_like(X)
    for (mx, my) in magnet_centers:
        phi += np.exp(-((X - mx) ** 2 + (Y - my) ** 2) / (2 * sigma ** 2))
    return phi


def laplacian(field):
    """Discrete Laplacian with periodic wrap (toy smoothing)."""
    return (
        np.roll(field, 1, axis=0) + np.roll(field, -1, axis=0)
        + np.roll(field, 1, axis=1) + np.roll(field, -1, axis=1)
        - 4 * field
    )


In [ ]:
def simulate(freq=120, depth_mm=6.0, alpha0=0.6, beta0=0.4, curing_minutes=8,
             magnet_pattern="double", seed=0):
    np.random.seed(seed)
    freq_norm = np.clip(freq / 200.0, 0.5, 3.0)
    depth_scale = np.clip(depth_mm / 6.0, 0.5, 1.5)

    # Convert to spatial frequencies (toy mapping)
    fx = 0.5 + freq_norm * 0.6
    fy = 0.5 + freq_norm * 0.4

    # Magnet layouts
    if magnet_pattern == "double":
        centers = [(0.3, 0.5), (0.7, 0.5)]
    elif magnet_pattern == "single":
        centers = [(0.5, 0.5)]
    elif magnet_pattern == "ladder":
        centers = [(0.2, 0.4), (0.4, 0.6), (0.6, 0.4), (0.8, 0.6)]
    else:  # random scatter
        centers = [(np.random.uniform(0.2, 0.8), np.random.uniform(0.3, 0.7)) for _ in range(4)]

    Phi_ac = acoustic_potential(fx, fy)
    Phi_mag = magnet_potential(centers, sigma=0.06)

    # Normalise potentials for stability
    Phi_ac /= np.max(np.abs(Phi_ac))
    Phi_mag /= np.max(np.abs(Phi_mag))

    steps = int(np.clip(curing_minutes * 8, 20, 150))
    dt = 0.1
    smooth_k = 0.04
    relax = 0.3

    h = np.zeros_like(X)
    history = []

    for step in range(steps):
        t_frac = step / steps
        alpha_t = alpha0 * (1 - t_frac)
        beta_t = beta0 * (1 - t_frac * 0.7)
        drive = depth_scale * alpha_t * Phi_ac + beta_t * Phi_mag
        h += dt * (drive - relax * h)
        h += smooth_k * laplacian(h)
        history.append(h.copy())

    return h, history, Phi_ac, Phi_mag, centers


In [ ]:
patterns = ["double", "single", "ladder", "random"]


def sandbox(freq=120, depth=6.0, alpha=0.6, beta=0.4, curing=8.0, pattern="double", seed=0):
    h, history, Phi_ac, Phi_mag, centers = simulate(
        freq=freq,
        depth_mm=depth,
        alpha0=alpha,
        beta0=beta,
        curing_minutes=curing,
        magnet_pattern=pattern,
        seed=seed,
    )

    fig, axes = plt.subplots(1, 3, figsize=(13, 4))

    im0 = axes[0].imshow(Phi_ac, cmap="coolwarm", origin="lower")
    axes[0].set_title("Φ_ac (sound)")
    plt.colorbar(im0, ax=axes[0], shrink=0.7)

    im1 = axes[1].imshow(Phi_mag, cmap="magma", origin="lower")
    axes[1].set_title("Φ_mag (magnets)")
    plt.colorbar(im1, ax=axes[1], shrink=0.7)

    im2 = axes[2].imshow(h, cmap="viridis", origin="lower")
    axes[2].set_title("Height proxy h(x,y)")
    plt.colorbar(im2, ax=axes[2], shrink=0.7)

    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])

    axes[2].text(0.02, 0.95, f"peak-to-peak ≈ {h.max() - h.min():.2f}", color="white", transform=axes[2].transAxes)

    plt.tight_layout()
    plt.show()


interact(
    sandbox,
    freq=FloatSlider(value=120, min=60, max=220, step=5, description="Hz"),
    depth=FloatSlider(value=6.0, min=3.0, max=12.0, step=0.5, description="depth mm"),
    alpha=FloatSlider(value=0.6, min=0.0, max=1.0, step=0.05, description="α sound"),
    beta=FloatSlider(value=0.4, min=0.0, max=1.0, step=0.05, description="β magnets"),
    curing=FloatSlider(value=8.0, min=2.0, max=15.0, step=0.5, description="curing min"),
    pattern=patterns,
    seed=IntSlider(value=0, min=0, max=50, step=1, description="seed"),
);


### Notes

- Toy dynamics exaggerate relief to highlight trends; expect real amplitudes of only a few millimetres.
- Start with `pattern="double"`, match magnet spacing to tray rails, then explore.
- Pull screenshots into `data/sample_results/` (ignored) for quick share-outs.
